<center><h2>Recommender Systems</h2></center>

We will investigate some gradient-based algorithms on the very well known matrix factorization problem which is the most prominent approach for build a Recommender Systems.

# Problem Formulation

The problem of matrix factorization for collaborative filtering captured much attention, especially after the [Netflix prize](https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf). The premise behind this approach is to approximate a large rating matrix $R$ with the multiplication of two low-dimensional factor matrices $P$ and $Q$, i.e. $R \approx \hat{R} = P^TQ$, that model respectively users and items in some latent space. For instance, matrix $R$ has dimension $m \times  n$ where $m$ and $n$ are restrictively the number of users and items, both large; while $P$ has size $m \times  k$ and contains user information in a latent space of size $k<<m,n$, $Q$ has size $n\times k$ and contains item information in the same latent space of size $k << m,n$. Typical values for $m, n$ are $10^6$ while $k$ is in the tens.

For a pair of user and item $(u_i,i_j)$ for which a rating $r_{ij}$ exists, a common approach approach is based on the minimization of the $\ell_2$-regularized quadratic error:
$$  \ell_{u_i,i_j}(P,Q)= \left(r_{ij} - p_{i}^{\top}q_{j}\right)^2 + \lambda(|| p_{i} ||^{2} + || q_{j} ||^2 )  $$
where $p_i$ is the column vector composed of the $i$-th line of $P$ and  $\lambda\geq 0$ is a regularization parameter. The whole matrix factorization problem thus writes
$$ \min_{P,Q} \sum_{i,j : r_{ij} \text{exists}}  \ell_{u_i,i_j}(P,Q). $$
Note that the error $ \ell_{u_i,i_j}(P,Q)$ depends only on $P$ and $Q$ through $p_{i}$ and $q_{j}$; however, item $i_j$ may also be rated by user $u_{i'}$ so that the optimal factor $q_{j}$ depends on both $p_{i}$ and $p_{i'}$.

In [4]:
# sc.stop()
# set up spark environment (Using Spark Local Mode)
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.setMaster("local[2]")
conf.setAppName("MSIAM part III - Matrix Factorization")

sc = SparkContext(conf = conf)

In [5]:
import numpy as np

def parseRating(line):
    fields = line.split('::')
    return int(fields[0]), int(fields[1]), float(fields[2])

# path to MovieLens dataset
movieLensHomeDir="../data/movielens/medium/"

# ratings is an RDD of (userID, movieID, rating)
ratingsRDD = sc.textFile(movieLensHomeDir + "ratings.dat").map(parseRating).setName("ratings").cache()

numRatings = ratingsRDD.count()
print(ratingsRDD.first())
numUsers = ratingsRDD.map(lambda r: r[0]).distinct().count()
numMovies = ratingsRDD.map(lambda r: r[1]).distinct().count()
print("We have %d ratings from %d users on %d movies.\n" % (numRatings, numUsers, numMovies))

N = ratingsRDD.map(lambda r: r[0]).max()
M = ratingsRDD.map(lambda r: r[1]).max()
matrixSparsity = float(numRatings)/float(M*N)
print("We have %d users, %d movies and the rating matrix has %f percent of non-zero value.\n" % (N, M, 100*matrixSparsity))

(1, 1193, 5.0)
We have 1000209 ratings from 6040 users on 3706 movies.

We have 6040 users, 3952 movies and the rating matrix has 4.190221 percent of non-zero value.



> Split (ramdomly) the dataset into training versus testing sample. We learn over 70% (for example) of the users, we test over the rest.

> Define a routine that returns the predicted rating from factor matrices. Form a RDD with the following elements `(i,j,true rating,predicted rating)`. 

> Define a routine that returns the Mean Square Error (MSE).


In [8]:

trainTestSplit = ratingsRDD.randomSplit(weights=[0.70, 0.30],seed=1)
train = trainTestSplit[0]
test = trainTestSplit[1]

# P -> ((i), ith row of P)
# Q -> ((j), jth col of Q)
# R -> ((i,j), true_rating_ij)
def predict_rating(P, Q, R, i, j):
    #print(P.lookup(i))
    #print(Q.lookup(j))
    predicted_rating = np.dot(P.lookup(i)[0], Q.lookup(j)[0])
    return sc.parallelize([((i,j), R.lookup((i,j))[0], predicted_rating)])
    
# R_hat -> ((i, j), true_rating, predicted_rating)
def mean_square_error(R_hat):
    return ((R_hat[1] - R_hat[2]) ** 2)

P = sc.parallelize([(1, np.array([1,1])), (2, np.array([2,2]))])
Q = sc.parallelize([(1, np.array([1,1])), (2, np.array([2,2]))])
R = sc.parallelize([((1,1), 1), ((2,2), 1), ((1,2), 1), ((2,1), 1)])

print(P.take(1))

print(mean_square_error(predict_rating(P, Q, R, 2, 1).take(1)[0]))

[(1, array([1, 1]))]
9


> Implementing a gradient algorithm in `Python` on the training set.  Taking a step size (learning rate) $\gamma=0.001$ and stop after a specified number of iterations. Investigate the latent space size with $K=2$).


Stochastic Gradient Descent (SGD) simply does away with the expectation in the update and computes the gradient of the parameters using only a single or a few training examples. 

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline

def prediction(P,Q):
    return np.dot(P,Q.T)

def rmse(R,Q,P):
    return np.sum(((R - prediction(P,Q)))**2)/len(R[R > 0])

train_errors = []
# PYTHON
# Beta is alpha/2
def full_grad_algo(R, P, Q, K, steps=10, alpha=0.001, beta=0.02):
    Q = Q.T
    for step in range(steps):
        print(step)
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        train_errors.append(e)
        if e < 0.001:
            break
    return P, Q.T

K = 2

inputV_filepath = movieLensHomeDir + "ratings.dat"
from scipy import sparse
def CreateMatrix(num_users, num_movies) :
    with open(inputV_filepath, 'r') as f_in :
        V = sparse.lil_matrix((num_users, num_movies))
        for line in f_in :
            line = line.rstrip()
            eachline = line.split("::")
            for i in range(3) :
                eachline[i] = int(eachline[i])
            V[eachline[0] - 1, eachline[1] - 1] = eachline[2]
    return V

V = CreateMatrix(N,M)
R = V.toarray()

steps = 5

P = np.random.rand(N,K)
Q = np.random.rand(M,K)

nP, nQ = full_grad_algo(R, P, Q, K, steps)
nR = np.dot(nP, nQ.T)

print(train_errors)

plt.plot(range(steps), train_errors, label='Training Data');
plt.title('SGD-WR Learning Curve')
plt.xlabel('Number of Epochs');
plt.ylabel('MSE');
plt.legend()
plt.grid()
plt.show()

KeyboardInterrupt: 

Running the ALS method already implemented in MLlib as a reference

In [7]:
from datetime import datetime
startTime = datetime.now()

movielens = sc.textFile("../data/movielens/medium/ratings.dat")

print(movielens.first()) #u'196\t242\t3\t881250949'
print(movielens.count()) #100000

#Clean up the data by splitting it
#Movielens readme says the data is split by tabs and
#is user product rating timestamp
clean_data = movielens.map(lambda x:x.split("::"))

#As an example, extract just the ratings to its own RDD
#rate.first() is 3
rate = clean_data.map(lambda y: int(y[2]))
print(rate.mean()) #Avg rating is 3.52986

#Extract just the users
users = clean_data.map(lambda y: int(y[0]))
print(users.distinct().count()) #943 users

#You don't have to extract data to its own RDD
#This command counts the distinct movies
#There are 1,682 movies
clean_data.map(lambda y: int(y[1])).distinct().count() 

# AVERAGE RATING - 3.58

#Need to import three functions / objects from the MLlib
from pyspark.mllib.recommendation\
    import ALS,MatrixFactorizationModel, Rating

#We'll need to map the movielens data to a Ratings object 
#A Ratings object is made up of (user, item, rating)
mls = movielens.map(lambda l: l.split('::'))
ratings = mls.map(lambda x: Rating(int(x[0]),\
    int(x[1]), float(x[2])))

#Need a training and test set
train, test = ratings.randomSplit([0.7,0.3],7856)

print(train.count()) #70,005
print(test.count()) #29,995

#Need to cache the data to speed up training
train.cache()
test.cache()

#Setting up the parameters for ALS
rank = 5 # Latent Factors to be made
numIterations = 10 # Times to repeat process
#Create the model on the training data
model = ALS.train(train, rank, numIterations)

#Examine the latent features for one product
print(model.productFeatures().first())
#(12, array('d', [-0.29417645931243896, 1.8341970443725586, 
    #-0.4908868968486786, 0.807500958442688, -0.8945541977882385]))

#Examine the latent features for one user
print(model.userFeatures().first())
#(12, array('d', [1.1348751783370972, 2.397622585296631,
    #-0.9957215785980225, 1.062819480895996, 0.4373367130756378]))

# For Product X, Find N Users to Sell To
model.recommendUsers(242,100)

# For User Y Find N Products to Promote
model.recommendProducts(196,10)

#Predict Single Product for Single User
model.predict(196, 242)

# Predict Multi Users and Multi Products
# Pre-Processing
pred_input = train.map(lambda x:(x[0],x[1]))   

# Lots of Predictions
#Returns Ratings(user, item, prediction)
pred = model.predictAll(pred_input) 

#Get Performance Estimate
#Organize the data to make (user, product) the key)
true_reorg = train.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred.map(lambda x:((x[0],x[1]), x[2]))

#Do the actual join
true_pred = true_reorg.join(pred_reorg)

#Need to be able to square root the Mean-Squared Error
from math import sqrt

MSE = true_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
RMSE = sqrt(MSE)#Results in 0.7629908117414474

#Test Set Evaluation
#More dense, but nothing we haven't done before
test_input = test.map(lambda x:(x[0],x[1])) 
pred_test = model.predictAll(test_input)
test_reorg = test.map(lambda x:((x[0],x[1]), x[2]))
pred_reorg = pred_test.map(lambda x:((x[0],x[1]), x[2]))
test_pred = test_reorg.join(pred_reorg)
test_MSE = test_pred.map(lambda r: (r[1][0] - r[1][1])**2).mean()
test_RMSE = sqrt(test_MSE)#1.0145549956596238
endTime = datetime.now()
diff = endTime - startTime
print("Root Mean Square Error on testing data : " + str(test_MSE))
print("Time taken on 2 cores : " + str(diff.total_seconds()) + " seconds.")

1::1193::5::978300760
1000209
3.58156445303
6040
699782
300427
(2, array('d', [-0.8554146885871887, -0.29376375675201416, 0.17143267393112183, -0.2958856523036957, -0.585989773273468]))
(2, array('d', [-2.3156867027282715, -1.8215765953063965, 0.15810754895210266, -0.27069902420043945, -1.3080871105194092]))
Root Mean Square Error on testing data : 0.775564392921
Time taken on 2 cores : 137.067409 seconds.


<small><i>Examples taken from the course [Convex and Distributed Optimization](http://www.iutzeler.org/CDO/) (2016).</i></small>